In [ ]:
# !pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [ ]:
import twint
import pathlib
import calendar
import datetime

# Twint fix
import nest_asyncio
nest_asyncio.apply()

In [ ]:
def info(msg):
    now = datetime.datetime.now()
    print(f"[{now.strftime('%Y%m%d-%H%M%S')}] {msg}")

def mkdir_if_not_exists(dir_name):
    directory = pathlib.Path(dir_name)
    directory.exists()
    directory.mkdir(parents = True, exist_ok = True)
    
def path_generatior(year, month, day, hashtag):
    return f"data/{year}/{month:02d}/{day:02d}/{year}-{month:02d}-{day:02d}-{hashtag.replace('#', '')}.pd"

def get_tweets_of_a_day(year, month, day, hashtag):
    info(f"Start collecting tweets for {year}/{month}/{day} for hashtag: {hashtag}")
    
    c = twint.Config()
    
    since = datetime.date(year=year, month=month, day=day)
    until = since + datetime.timedelta(days=1)

    c.Search = hashtag
    c.Limit = 20
    c.Since = since.strftime('%Y-%m-%d')
    c.Until = until.strftime('%Y-%m-%d')
    c.Geo = "39.6459604,-99.1227847,1400km"
    c.Filter_retweets = True
    c.Location = True
    c.Retweets = False
    c.User_full = True
    c.Hide_output = True
    c.Pandas = True
    
    info("Searching...")
    twint.run.Search(c)
    
    df = twint.storage.panda.Tweets_df
    
    info("Storing...")
    path = path_generatior(year, month, day, hashtag)
    df.reset_index().to_feather(path)
    
HASHTAGS = [
    '#GlobalWarming',
    '#ClimateChange',
]

mkdir_if_not_exists('data')

for year in range(2021, 2013, -1):
    mkdir_if_not_exists(f"data/{year}")
    
    for month in range(1, 13):
        mkdir_if_not_exists(f"data/{year}/{month:02d}")
        
        for day in range(1, calendar.monthrange(year, month)[1]+1, 7):
            mkdir_if_not_exists(f"data/{year}/{month:02d}/{day:02d}")
            
            for hashtag in HASHTAGS:
                if pathlib.Path(path_generatior(year, month, day, hashtag)).is_file():
                    continue
                    
                get_tweets_of_a_day(year, month, day, hashtag)

            break
        break
    break